# Juliaによる非線形方程式の全解探索アルゴリズムの実装

## 全解探索アルゴリズムのソースコード

In [23]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function orig_is_subset(X,Y)
    for i = 1:size(X)[1]
        if !( issubset(X[i],Y[i]) )
            return false
        end
    end
    return true
end

function orig_intersect(X,Y)
    L = X*1
    for i = 1:size(X)[1]
        L[i]= intersect(X[i],Y[i])
    end
    return L
end

function orig_is_empty(X,Y)
    for i = 1:size(X)[1]
        if ( isempty(intersect(X[i],Y[i])) )
            return true
        end
    end
    return false
end

function orig_devide(X)
    #最も長い辺を二等分する
    max_diam=0
    max_index=0
    min_diam=diam(X[1])
    min_index=1
    X1,X2=[],[]
    for (i,x) in enumerate(X)
        if diam(x)>max_diam
            max_diam=diam(x)
            max_index=i
        end
        if diam(x)<min_diam
            min_diam=diam(x)
            min_index=i
        end
    end

    if diam(X[min_index])<10^(-8)
        return false
    end
    
    X1=X*1 #値をコピーする
    X2=X*1 #値をコピーする
    X1[max_index]= intersect(X[max_index],X[max_index]-radius(X[max_index]))
    X2[max_index]= intersect(X[max_index],X[max_index]+(diam(X[max_index])-radius(X[max_index])))
    return X1,X2
end

function border(K,I)
    c=0.9
    K_width=diam.(K)
    I_width=diam.(I)
    max=0
    for i=1:size(K)[1]
        if (K_width[i]/I_width[i])>max
            max=K_width[i]/I_width[i]
        end
    end
    if max<=0.9
        return true
    else
        return false
    end
end

function remove_duplication(S)
    tmp_index=[]
    new_s=[]
    for i=1:size(S)[1]-1
        for j = i+1:size(S)[1]
            if !orig_is_empty(S[i],S[j])
                
                if (i in tmp_index) || (j in tmp_index)
                    push!(tmp_index,i)
                    push!(tmp_index,j)
                    continue
                end
                push!(tmp_index,i)
                push!(tmp_index,j)
                I = orig_intersect(S[i],S[j])
                push!(new_s,I)
            end
        end
    end
    for i=1:size(S)[1]
        if !(i in tmp_index)
            push!(new_s,S[i])
        end
    end
    
    return new_s
end


function allsol(F,X)
    dimention = size(X)[1]
    
    L = [X] #調査する区間のリスト
    S = Array[] #見つかった解のリスト
    
    cnt=0
    
    while (size(L)[1] != 0)
        cnt+=1
        if cnt >100
            return false
        end
        
        X = popfirst!(L) #Lから区間を一つ取り出す
        
        #非存在判定
        if !( orig_is_subset([(0. .. 0.) for i=1:dimention], F(X)) ) 
            continue
        end
        
        c = mid.(X)
        j = ForwardDiff.jacobian(F,X)
        
        #非存在判定
        if !( orig_is_subset([(0. .. 0.) for i=1:dimention], F(c)+j*(X-c)) )
            continue
        end
        
        R = []
        #逆行列計算
        try
            R = inv(ForwardDiff.jacobian(F,c))
        catch e
            #逆行列計算に失敗したら区間を分割
            try
                X1,X2 = orig_devide(X)
                push!(L, X1)
                push!(L, X2)
            catch 
                continue
            end
            continue
        end
        
        M = Matrix{Float64}(I,size(R))-R*j
        K = c -R*F(c)+M*(X-c)
        
        #解なし
        if (orig_is_empty(K,X))
            continue 
        end
        
        #解あり
        if (orig_is_subset(K,X))　
            push!(S, K) #解のリストに追加
            continue
        end
        
        # 境界に乗ってた時
        if border(K,X)
            push!(L,K)
            continue
        end
        
        #分割して候補者区間に追加
        try
            X1,X2 = orig_devide(orig_intersect(K,X))
            #K and X を2分割
            push!(L, X1)
            push!(L, X2)
            continue
        catch
            continue
        end
    end
    
    #重複を排除
    S = remove_duplication(S)
    
    #区間を狭める
    for i =1:size(S)[1]
        si=S[i]
        while(maximum(radius,si) >= 10^-8)
            j=0
            try
                j = ForwardDiff.jacobian(F,si)
            catch
                break
            end
            c = mid.(si)
            R=0
            try 
                R = inv(ForwardDiff.jacobian(F,c)) 
            catch
                break
            end
            M = Matrix{Float64}(I,size(R))-R*j
            k = c -R*F(c)+M*(si-c)
            if orig_is_subset(k,si)
                si=k
            else
                break
            end
        end
        S[i]=si
    end
    
    setformat(:full)
    for s=1:size(S)[1] 
        print("解");print(s);print(": ");
        for i in S[s]
            print(i)
            print(" ")
        end
        println("")
    end
    
    return S
end

# 重解　＝＞区間幅に制限をつける
# 高速化？

allsol (generic function with 1 method)

## 検証0

In [22]:
a=[2 5 ;3 1]
f=[x1+x2+2 ;x1^3-x2^2+2]

F((x1,x2))=a*f

@show F((1,2))
allsol(F,X)

F((1, 2)) = [18, 14]


Any[]

## 検証1

In [30]:
X = [(-10. ..10.),(-10. ..10.),(-10. ..10.),(-10. ..10.)]

rb = 10^4
rc= 5*10^3
vcc=-5
af=0.99
ar=0.5
is=10^(-9)
vt=0.053
vs=-0.64
gb=1. /rb
gc=1. /rc

a=[
    1 -ar 0 0;
    -ar 1 0 0;
    0 0 1 -ar;
    0 0 -ar 1
]

b(v1,v2,v3,v4)=[
    (is/af)*(exp(v1/vt)-1);
    (is/ar)*(exp(v2/vt)-1);
    (is/af)*(exp(v3/vt)-1);
    (is/ar)*(exp(v4/vt)-1)
]
c=[
    2*gb+gc -(gb+gc) -2gb gb;
    -(gb+gc) gb+gc gb 0;
    -2*gb gb 2*gb +gc -(gb+gc);
    gb 0 -(gb+gc) gb+gc
]
v(v1,v2,v3,v4) = [v1;v2;v3;v4]
d=[
    gc*vcc;gb*vs-gc*vcc;gc*vcc;gb*vs-gc*vcc
]

F1((v1, v2,v3,v4))=a*b(v1, v2,v3,v4)+c*v(v1, v2,v3,v4)+d
@show allsol(F1,X)


LoadError: ArgumentError: row 3 has mismatched number of columns (expected 4, got 5)